### How to price Balloon Mortgage

In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
from absbox import API

#localAPI = API("http://localhost:8081",lang='english',check=False)
localAPI = API("https://absbox.org/api/dev",lang='english',check=False)

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.28.7, server:0.28.18

In [13]:
mortgage = ["Mortgage"
            ,{"originBalance": 12000.0
              ,"originRate": ["fix",0.045]
              ,"originTerm": 60
              ,"freq": "Monthly"
              ,"type": ("Balloon",120)
              ,"originDate": "2021-02-01"}
            ,{"currentBalance": 12000.0
              ,"currentRate": 0.075
              ,"remainTerm": 12
              ,"status": "Current"}]

#### Typical Cashflow Profile for Balloon Mortgage

We model the balloon mortgage via change `type` to `("Balloon, xxx)`

Then the mortgage will amortized as if it has a term `xxx` and have a final principal repayment at last period

In [14]:
p = localAPI.runAsset("2021-02-01"
                      ,[mortgage]
                      ,poolAssump=("Pool"
                                    ,("Mortgage",None ,None, None, None)
                                     ,None
                                     ,None)
                      ,read=True)
p[0]

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2025-02-01,12000.00,0.00,0.00,0,0,0,0,0.075,None,None,0.00,0,0,0,0,0
2025-03-01,11867.52,132.48,75.00,0,0,0,0,0.075,None,None,132.48,0,0,0,0,0
2025-04-01,11734.21,133.31,74.17,0,0,0,0,0.075,None,None,265.79,0,0,0,0,0
2025-05-01,11600.06,134.15,73.33,0,0,0,0,0.075,None,None,399.94,0,0,0,0,0
2025-06-01,11465.08,134.98,72.50,0,0,0,0,0.075,None,None,534.92,0,0,0,0,0
2025-07-01,11329.25,135.83,71.65,0,0,0,0,0.075,None,None,670.75,0,0,0,0,0
2025-08-01,11192.57,136.68,70.80,0,0,0,0,0.075,None,None,807.43,0,0,0,0,0
2025-09-01,11055.04,137.53,69.95,0,0,0,0,0.075,None,None,944.96,0,0,0,0,0
2025-10-01,10916.65,138.39,69.09,0,0,0,0,0.075,None,None,1083.35,0,0,0,0,0


#### When "CDR" is failing

`CDR` is a constant annualized default rates which apply default amounts on each payment period.

The `CDR` works if the monthly payment are evenly distributed, because the payment obligation remains same during the life time of a mortgage.


The issue is, for `Balloon Mortgage` , does the last payment which has a larger portion share same default rate with previous payments ? 

Imagine a borrower has obligation of paying:

* 10 dollars per month and 10000 dollars at last payment

The lender shall expect a larger risk at the last payment from the borrower, in other term, a larger default risk. In such case, the `CDR` way may doesn't present a pefect fit.

In [15]:
p = localAPI.runAsset("2021-02-01"
                      ,[mortgage]
                      ,poolAssump=("Pool"
                                    ,("Mortgage",{"CDR":0.05} ,None, None, None)
                                     ,None
                                     ,None)
                      ,read=True)
p[0]

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2025-02-01,12000.00,0.00,0.00,0,0.00,0,0.00,0.075,None,None,0.00,0,0,0.00,0,0.00
2025-03-01,11820.92,131.96,74.70,0,47.12,0,47.12,0.075,None,None,131.96,0,0,47.12,0,47.12
2025-04-01,11637.33,132.21,73.55,0,51.38,0,51.38,0.075,None,None,264.17,0,0,98.50,0,98.50
2025-05-01,11455.90,132.48,72.42,0,48.95,0,48.95,0.075,None,None,396.65,0,0,147.45,0,147.45
2025-06-01,11273.38,132.73,71.28,0,49.79,0,49.79,0.075,None,None,529.38,0,0,197.24,0,197.24
2025-07-01,11092.97,132.99,70.16,0,47.42,0,47.42,0.075,None,None,662.37,0,0,244.66,0,244.66
2025-08-01,10911.50,133.25,69.02,0,48.22,0,48.22,0.075,None,None,795.62,0,0,292.88,0,292.88
2025-09-01,10730.58,133.49,67.90,0,47.43,0,47.43,0.075,None,None,929.11,0,0,340.31,0,340.31
2025-10-01,10551.68,133.76,66.78,0,45.14,0,45.14,0.075,None,None,1062.87,0,0,385.45,0,385.45


#### Solution from Absbox/Hastructure


A new assumption with 2 default rates is introduced by (Hastructure: `0.28.18`,/Absbox: `0.28.14`), 

`{"DefaultAtEndByRate":(A,B)}`

* `A` : a CDR will be applied for payment periods except last period
  * default rate = days between ( current period , last period), then annualized with CDR
* `B` : a CDR will be applied for last payment period
  * default rate = days between ( current period , BEGIN period), then annualized with CDR

Now we can expect a larger portion of default in last period which factoring the life time span. ( The longer life of mortgage, the larger default rate)

In [16]:
mortgagePoolAssump = ("Pool"
                        ,("Mortgage",{"DefaultAtEndByRate":(0.05,0.05)} ,None, None, None)
                         ,None
                         ,None)

p = localAPI.runAsset("2021-02-01"
                      ,[mortgage]
                      ,poolAssump=mortgagePoolAssump
                      ,read=True)
p[0]

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2025-02-01,12000.00,0.00,0.00,0,0.00,0,0.00,0.075,None,None,0.00,0,0,0.00,0,0.00
2025-03-01,11820.92,131.96,74.70,0,47.12,0,47.12,0.075,None,None,131.96,0,0,47.12,0,47.12
2025-04-01,11637.33,132.21,73.55,0,51.38,0,51.38,0.075,None,None,264.17,0,0,98.50,0,98.50
2025-05-01,11455.90,132.48,72.42,0,48.95,0,48.95,0.075,None,None,396.65,0,0,147.45,0,147.45
2025-06-01,11273.38,132.73,71.28,0,49.79,0,49.79,0.075,None,None,529.38,0,0,197.24,0,197.24
2025-07-01,11092.97,132.99,70.16,0,47.42,0,47.42,0.075,None,None,662.37,0,0,244.66,0,244.66
2025-08-01,10911.50,133.25,69.02,0,48.22,0,48.22,0.075,None,None,795.62,0,0,292.88,0,292.88
2025-09-01,10730.58,133.49,67.90,0,47.43,0,47.43,0.075,None,None,929.11,0,0,340.31,0,340.31
2025-10-01,10551.68,133.76,66.78,0,45.14,0,45.14,0.075,None,None,1062.87,0,0,385.45,0,385.45
